<h1> Arabic letters and diacritics

In [79]:
import pickle
import re
import pyarabic.araby as araby
import pyarabic.number as number

In [80]:
arabic_letters = []
diacritics = []
diacritics_to_id = {}
with open('./utils/arabic_letters.pickle', 'rb') as f:
    arabic_letters = pickle.load(f)
with open('./utils/diacritics.pickle', 'rb') as f:
    diacritics = pickle.load(f)
with open('./utils/diacritic2id.pickle', 'rb') as f:
    diacritics_to_id = pickle.load(f)

print(arabic_letters)
print(diacritics)
print(diacritics_to_id)

{'ل', 'ظ', 'ث', 'أ', 'ي', 'ك', 'ء', 'غ', 'ر', 'ن', 'ة', 'ق', 'س', 'ا', 'ج', 'ص', 'ئ', 'ت', 'م', 'د', 'و', 'ع', 'خ', 'ذ', 'إ', 'ى', 'ب', 'ز', 'ض', 'ف', 'آ', 'ح', 'ه', 'ؤ', 'ش', 'ط'}
{'َ', 'ّ', 'ٌ', 'ٍ', 'ْ', 'ُ', 'ً', 'ِ'}
{'َ': 0, 'ً': 1, 'ُ': 2, 'ٌ': 3, 'ِ': 4, 'ٍ': 5, 'ْ': 6, 'ّ': 7, 'َّ': 8, 'ًّ': 9, 'ُّ': 10, 'ٌّ': 11, 'ِّ': 12, 'ٍّ': 13, '': 14}


<h2> Read train and val data

In [81]:
train_data = []
val_data = []
with open('./data/train.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        train_data.append(line.strip())
with open('./data/val.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        val_data.append(line.strip())
print(len(train_data))
print(len(val_data))

50000
2500


<h2> Clean data

In [82]:
def clean_arabic_text(text):
    """
    Clean text to keep only Arabic letters, diacritics, and spaces
    """
    # Create a set of allowed characters (Arabic letters + diacritics + space)
    allowed_chars = arabic_letters.union(diacritics, {' ', '\t', '\n'})
    
    # Filter the text to keep only allowed characters
    cleaned_text = ''.join(char for char in text if char in allowed_chars)
    
    # Normalize whitespace
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    
    return cleaned_text

In [83]:
for i in range(len(train_data)):
    train_data[i] = clean_arabic_text(train_data[i])
for i in range(len(val_data)):
    val_data[i] = clean_arabic_text(val_data[i])

<h2> Tokenization

In [ ]:
tokenized_train_data = []
for sentence in train_data:
    tokenized_train_data.append(araby.tokenize(sentence))

['وَلَوْ', 'جَمَعَ', 'ثُمَّ', 'عَلِمَ', 'تَرْكَ', 'رُكْنٍ', 'مِنْ', 'الْأُولَى', 'بَطَلَتَا', 'وَيُعِيدُهُمَا', 'جَامِعًا', 'أَوْ', 'مِنْ', 'الثَّانِيَةِ', 'فَإِنْ', 'لَمْ', 'يَطُلْ', 'تَدَارَكَ', 'وَإِلَّا', 'فَبَاطِلَةٌ', 'وَلَا', 'جَمَعَ', 'وَلَوْ', 'جَهِلَ', 'أَعَادَهُمَا', 'لِوَقْتَيْهِمَا']
